In [ ]:
# import libraries
import requests
import os
import hashlib
import pathlib
import json
from datetime import datetime
import pytz
from urllib.parse import urlparse
import time
import pandas as pd
import googlemaps
import concurrent.futures
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt
import folium
import statsmodels.api as sm
from statsmodels.formula.api import glm
from statsmodels.discrete.count_model import ZeroInflatedNegativeBinomialP
import numpy as np

In [ ]:
# import functions
from functions.routes_api_functions import Location
from functions.routes_api_functions import calculateRoute
from functions.routes_api_functions import parseDataframeToLocations
from functions.routes_api_functions import createDataFrameFromResults
from functions.routes_api_functions import cache_file_path
from functions.routes_api_functions import save_to_cache
from functions.routes_api_functions import load_from_cache
from functions.routes_api_functions import fetch_all_routes
from functions.routes_api_functions import create_departure_time

## Routes API Data Generate

In [ ]:
# import data
od = pd.read_csv('../Data/OD3am_10000.csv')

In [ ]:
od

In [ ]:
origins = od[['oid', 'oy', 'ox']]
destinations = od[['did', 'dy', 'dx']]

In [ ]:
# convert origins, destinations to geodataframe
origins = gpd.GeoDataFrame(origins, geometry=[Point(xy) for xy in zip(origins.ox, origins.oy)])
destinations = gpd.GeoDataFrame(destinations, geometry=[Point(xy) for xy in zip(destinations.dx, destinations.dy)])

In [ ]:
# apply parsedataframetoLocations
originsLoc = parseDataframeToLocations(origins)
destinationsLoc = parseDataframeToLocations(destinations)

In [ ]:
# api key
key = 'Jessica API Key'

In [ ]:
# calculate routes
# Jan 17 (Wednesday) 3AM
departure_times = [(2024, 1, 17, hour, 0, 0) for hour in od['hod']]
# use departure_times list as input
results = fetch_all_routes(apikey=key, origins=originsLoc, destinations=destinationsLoc, traffic='TRAFFIC_AWARE', departure_times=departure_times)

In [ ]:
results

In [ ]:
for index, item in enumerate(results):
    if not item:  # Checks if the route object is empty
        print(f"Empty route object found at index {index}")

In [ ]:
results[17]

In [ ]:
df = createDataFrameFromResults(results)
df

In [ ]:
# remove no results O-D
remove = [17, 840, 2179, 3795, 4380, 5747, 8436, 8517]
od2 = od.drop(remove)

In [ ]:
od2 = od2.reset_index(drop=True)
df = df.reset_index(drop=True)

In [ ]:
len(od2)

In [ ]:
len(df)

In [ ]:
combined_df = pd.concat([od2, df], axis=1)

In [ ]:
combined_df

In [ ]:
# import uber data
uber = pd.read_csv('../Data/route_result_uber_hod3am_10000_freeflow.csv')

In [ ]:
uber

In [ ]:
# combine uber, routes api, free-flow data

In [ ]:
remove = [17, 840, 2179, 3795, 4380, 5747, 8436, 8517]
uber2 = uber.drop(remove)

In [ ]:
uber2 = uber2.reset_index(drop=True)

In [ ]:
uber2

In [ ]:
test_data = uber2
test_data['distance'] = combined_df['distance']
test_data['duration'] = combined_df['duration']
test_data['polyline'] = combined_df['polyline']

In [ ]:
test_data

In [ ]:
test_data.columns

In [ ]:
# outcome variable
test_data['uber_diff'] = test_data['uber_time'] - test_data['freeflow_travel_time']
test_data['gg_diff'] = test_data['duration'] - test_data['freeflow_travel_time']

## Check Statistics of Penalty Elements

In [ ]:
# mean
test_data[['freeflow_travel_time', 'freeflow_signal_count', 'freeflow_stop_count',
       'freeflow_crossing_count', 'freeflow_give_way_count',
       'freeflow_mini_roundabout_count', 'freeflow_left_count',
       'freeflow_right_count', 'freeflow_u_count']].mean()

In [ ]:
# variance
test_data[['freeflow_travel_time', 'freeflow_signal_count', 'freeflow_stop_count',
       'freeflow_crossing_count', 'freeflow_give_way_count',
       'freeflow_mini_roundabout_count', 'freeflow_left_count',
       'freeflow_right_count', 'freeflow_u_count']].var()

## Negative Binomial Regression

In [ ]:
# Negative Binomial Regression model without intercept - Google Time
formula = 'duration ~ 0 + freeflow_travel_time + freeflow_signal_count + freeflow_stop_count + freeflow_crossing_count + freeflow_give_way_count + freeflow_mini_roundabout_count + freeflow_left_count + freeflow_right_count + freeflow_u_count'
model = sm.GLM.from_formula(formula, data=test_data, family=sm.families.NegativeBinomial()).fit()

# Print Model
print(model.summary())

In [ ]:
# Negative Binomial Regression model without intercept - Uber Time
formula = 'uber_time ~ 0 + freeflow_travel_time + freeflow_signal_count + freeflow_stop_count + freeflow_crossing_count + freeflow_give_way_count + freeflow_mini_roundabout_count + freeflow_left_count + freeflow_right_count + freeflow_u_count'
model = sm.GLM.from_formula(formula, data=test_data, family=sm.families.NegativeBinomial()).fit()

# Print Model
print(model.summary())

## Poisson Regression

In [ ]:
# Poisson Regression - Google Time
formula = 'duration ~ 0 + freeflow_travel_time + freeflow_signal_count + freeflow_stop_count + freeflow_crossing_count + freeflow_give_way_count + freeflow_mini_roundabout_count + freeflow_left_count + freeflow_right_count + freeflow_u_count'
model = sm.GLM.from_formula(formula, data=test_data, family=sm.families.Poisson()).fit()

# Print Model
print(model.summary())

In [ ]:
# Negative Binomial Regression model without intercept - Uber Time
formula = 'uber_time ~ 0 + freeflow_travel_time + freeflow_signal_count + freeflow_stop_count + freeflow_crossing_count + freeflow_give_way_count + freeflow_mini_roundabout_count + freeflow_left_count + freeflow_right_count + freeflow_u_count'
model = sm.GLM.from_formula(formula, data=test_data, family=sm.families.Poisson()).fit()

# Print Model
print(model.summary())

## Zero Inflated Negative Binomial Regression

In [ ]:
# Zero-Inflated Negative Binomial Regression model without intercept - Google Time
formula = 'duration ~ 0 + freeflow_travel_time + freeflow_signal_count + freeflow_stop_count + freeflow_crossing_count + freeflow_give_way_count + freeflow_mini_roundabout_count + freeflow_left_count + freeflow_right_count + freeflow_u_count'
model = sm.ZeroInflatedNegativeBinomialP.from_formula(formula, data=test_data).fit()

# Print Model
print(model.summary())

In [ ]:
# Zero-Inflated Negative Binomial Regression model without intercept - Google Time
formula = 'uber_time ~ 0 + freeflow_travel_time + freeflow_signal_count + freeflow_stop_count + freeflow_crossing_count + freeflow_give_way_count + freeflow_mini_roundabout_count + freeflow_left_count + freeflow_right_count + freeflow_u_count'
model = sm.ZeroInflatedNegativeBinomialP.from_formula(formula, data=test_data).fit()

# Print Model
print(model.summary())

## Zero Inflated Poisson Regression

In [ ]:
# Zero-Inflated Negative Binomial Regression model without intercept - Google Time
formula = 'duration ~ 0 + freeflow_travel_time + freeflow_signal_count + freeflow_stop_count + freeflow_crossing_count + freeflow_give_way_count + freeflow_mini_roundabout_count + freeflow_left_count + freeflow_right_count + freeflow_u_count'
model = sm.ZeroInflatedPoisson.from_formula(formula, data=test_data).fit()

# Print Model
print(model.summary())

In [ ]:
# Zero-Inflated Negative Binomial Regression model without intercept - Google Time
formula = 'uber_time ~ 0 + freeflow_travel_time + freeflow_signal_count + freeflow_stop_count + freeflow_crossing_count + freeflow_give_way_count + freeflow_mini_roundabout_count + freeflow_left_count + freeflow_right_count + freeflow_u_count'
model = sm.ZeroInflatedPoisson.from_formula(formula, data=test_data).fit()

# Print Model
print(model.summary())